In [14]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
from dwave.samplers import SimulatedAnnealingSampler
import dimod
import psutil
import time
from scipy.sparse import lil_matrix, csc_matrix, block_diag




In [15]:
N_MODULES = 5
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX] * N_MODULES,
    ly=[LY] * N_MODULES,
    z=[i + Z_SPACING for i in range(N_MODULES)]
)

generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi / 6
)

N_PARTICLES = 100
event = generator.generate_event(N_PARTICLES)
event.hits

[Hit(hit_id=0, x=-0.12460871526485641, y=-0.3620273179599082, z=1.0, module_id=0, track_id=0),
 Hit(hit_id=5, x=0.3329323235555085, y=-0.4257786817364716, z=1.0, module_id=0, track_id=1),
 Hit(hit_id=10, x=0.2083722620309784, y=0.002905228211918778, z=1.0, module_id=0, track_id=2),
 Hit(hit_id=15, x=0.4041750900780867, y=0.32655351181953934, z=1.0, module_id=0, track_id=3),
 Hit(hit_id=20, x=-0.33387630716641187, y=0.3330590117861515, z=1.0, module_id=0, track_id=4),
 Hit(hit_id=25, x=0.45497925239249926, y=-0.3294636727066543, z=1.0, module_id=0, track_id=5),
 Hit(hit_id=30, x=0.05381539532591119, y=0.43235152439209856, z=1.0, module_id=0, track_id=6),
 Hit(hit_id=35, x=0.3336055172280513, y=-0.3917603155240042, z=1.0, module_id=0, track_id=7),
 Hit(hit_id=40, x=0.164238586596836, y=0.00171692712283014, z=1.0, module_id=0, track_id=8),
 Hit(hit_id=45, x=0.12630561452092973, y=-0.14234667720930258, z=1.0, module_id=0, track_id=9),
 Hit(hit_id=50, x=-0.09303759591532333, y=0.25329591450

In [16]:
params = {
    'alpha': 1.0,
    'beta': 1.0,
    'lambda': 100.0,} 

def generate_hamiltonian_optimized(event, params):
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    modules = sorted(event.modules, key=lambda module: module.z)

    
    segments = [
        em.segment(from_hit, to_hit)
        for idx in range(len(modules) - 1)
        for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)
    ]

    N = len(segments)  

    #initialize sparse block matrices for effviciency
    A_ang_blocks = []
    A_bif_blocks = []
    A_inh_blocks = []
    b = np.zeros(N)

    #total Hamiltonian into smaller blocks->> better for memory
    block_size = 500  
    num_blocks = (N + block_size - 1) // block_size  

    for block_idx in range(num_blocks):
        start_idx = block_idx * block_size
        end_idx = min(start_idx + block_size, N)

        #lil_matrix for each block
        A_ang_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_bif_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_inh_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)

        #filling of papricas
        for i in range(start_idx, end_idx):
            seg_i = segments[i]
            vect_i = seg_i.to_vect()
            norm_i = np.linalg.norm(vect_i)

            for j in range(i + 1, end_idx):  #
                seg_j = segments[j]
                vect_j = seg_j.to_vect()
                norm_j = np.linalg.norm(vect_j)

                cosine = np.dot(vect_i, vect_j) / (norm_i * norm_j)
                if np.abs(cosine - 1) < 1e-9:
                    A_ang_block[i - start_idx, j - start_idx] = 1
                    A_ang_block[j - start_idx, i - start_idx] = 1  # Symmetry with positive sign

                if seg_i.from_hit == seg_j.from_hit and seg_i.to_hit != seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                if seg_i.from_hit != seg_j.from_hit and seg_i.to_hit == seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                s_ab = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)
                if s_ab > 0:
                    A_inh_block[i - start_idx, j - start_idx] = beta * s_ab * s_ab
                    A_inh_block[j - start_idx, i - start_idx] = beta * s_ab * s_ab  # Symmetry with positive sign

        A_ang_blocks.append(A_ang_block)
        A_bif_blocks.append(A_bif_block)
        A_inh_blocks.append(A_inh_block)

    # combine withblock diagonal
    A_ang = block_diag(A_ang_blocks, format='csc')
    A_bif = block_diag(A_bif_blocks, format='csc')
    A_inh = block_diag(A_inh_blocks, format='csc')

    A = -1 * (A_ang + A_bif + A_inh)

    return A, b, segments

#performance measurement
process = psutil.Process()

start_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
start_time = time.time()

A, b, segments = generate_hamiltonian_optimized(event, params)
end_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
end_time = time.time()

memory_used = end_memory - start_memory
time_taken = end_time - start_time

print(f"Memory used: {memory_used:.2f} MB")
print(f"Time taken: {time_taken:.6f} seconds")

Memory used: 118.02 MB
Time taken: 173.349172 seconds


In [17]:
import dimod
import networkx as nx
import psutil
import time
from scipy.sparse import csc_matrix
from dwave.samplers import SimulatedAnnealingSampler
import numpy as np

def qubo_to_graph(A):
    G = nx.Graph()
    row, col = A.nonzero()
    for i, j in zip(row, col):
        if i != j:
            G.add_edge(i, j, weight=A[i, j])
    return G

def solve_sub_qubo(A_sub, b_sub):

    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    bqm.add_variables_from({i: b_sub[i] for i in range(len(b_sub))})

    row, col = A_sub.nonzero()

    for i, j in zip(row, col):
        if i != j:
            bqm.add_interaction(i, j, A_sub[i, j])

    sampler = SimulatedAnnealingSampler()
    response = sampler.sample(bqm, num_reads=100)

    best_sample = response.first.sample
    sol_sample = np.fromiter(best_sample.values(), dtype=int)

    return sol_sample

def decompositionsolver(A, b):

    process = psutil.Process()
    start_memory = process.memory_info().rss / (1024 ** 2)  
    start_time = time.time()

    G = qubo_to_graph(A)

    components = list(nx.connected_components(G))
    global_solution = np.zeros(A.shape[0], dtype=int)

    #solve each subqubo
    for component in components:
        
        indices = list(component)
        A_sub = A[indices, :][:, indices]
        b_sub = b[indices]

       
        sub_solution = solve_sub_qubo(A_sub, b_sub)
        
        global_solution[indices] = sub_solution

    
    end_memory = process.memory_info().rss / (1024 ** 2)  
    end_time = time.time()

    memory_used = end_memory - start_memory
    time_taken = end_time - start_time

    print(f"Solution sample: {global_solution}")
    print(f"Memory used: {memory_used:.2f} MB")
    print(f"Time taken: {time_taken:.6f} seconds")

    return global_solution

A = csc_matrix(A)  
b = np.array(b)    
decompositionsolver(A, b)



Solution sample: [1 0 0 ... 0 0 0]
Memory used: 707.83 MB
Time taken: 564.757709 seconds


array([1, 0, 0, ..., 0, 0, 0])